In [2]:
## Source: https://edwinhu.github.io/pin/


import os
import datetime

import numpy as np
from scipy.special import gammaln
import pandas as pd
from numpy import log, exp
import wrds
from pandas import DataFrame

try:
    from itertools import imap
except ImportError:
    imap = map

# optimization
from numba import jit


def lfact(x):
    """Compute the log factorial using the scipy gammaln function.

    This is commonly referred to as Stirlings approximation/formula for factorials."""
    return gammaln(x + 1)


def nanexp(x):
    """Computes the exponential of x, and replaces nan and inf with finite numbers.

    Returns an array or scalar replacing Not a Number (NaN) with zero, (positive) infinity with a very large number and negative infinity with a very small (or negative) number."""
    return np.nan_to_num(np.exp(x))

In [3]:
# GPIN method
def _lf_gpin(th_b, th_s, r, p, n_buys, n_sells, pdenom=1):
    res = log(th_b) * n_buys + log(1 - th_s) * n_sells - lfact(n_buys) - lfact(n_sells) - gammaln(r) + log(
        1 - p) * r + log(p) * (n_buys + n_sells) + gammaln(r + n_buys + n_sells) - log(pdenom) * r - log(pdenom) * (
                  n_buys + n_sells)
    return res


def _ll_gpin(a, r, p, eta, d, th, n_buys, n_sells):
    return np.array([log(1 - a) + _lf_gpin(th, th, r, p, n_buys, n_sells),
                     log(a * d) + _lf_gpin(th + eta, th, r, p, n_buys, n_sells, 1 + eta * p),
                     log(a * (1 - d)) + _lf_gpin(th, th - eta, r, p, n_buys, n_sells, 1 + eta * p)])


def compute_alpha_gpin(a, r, p, eta, d, th, n_buys, n_sells):
    """Compute the conditional alpha given parameters, buys, and sells.

    """
    ys = _ll_gpin(a, r, p, eta, d, th, n_buys, n_sells)

    ymax = ys.max(axis=0)
    lik = exp(ys - ymax)
    alpha = lik[1:].sum(axis=0) / lik.sum(axis=0)

    return alpha

In [13]:
# OWR calculation
def det3_owr(a):
    return (a[0][0] * (a[1][1] * a[2][2] - a[2][1] * a[1][2])
            - a[1][0] * (a[0][1] * a[2][2] - a[2][1] * a[0][2])
            + a[2][0] * (a[0][1] * a[1][2] - a[1][1] * a[0][2]))


def inv3_owr(a):
    invdet = 1 / det3_owr(a)
    m = np.zeros((3, 3))
    m[0, 0] = a[1, 1] * a[2, 2] - a[2, 1] * a[1, 2]
    m[0, 1] = a[0, 2] * a[2, 1] - a[0, 1] * a[2, 2]
    m[0, 2] = a[0, 1] * a[1, 2] - a[0, 2] * a[1, 1]
    m[1, 0] = a[1, 2] * a[2, 0] - a[1, 0] * a[2, 2]
    m[1, 1] = a[0, 0] * a[2, 2] - a[0, 2] * a[2, 0]
    m[1, 2] = a[1, 0] * a[0, 2] - a[0, 0] * a[1, 2]
    m[2, 0] = a[1, 0] * a[2, 1] - a[2, 0] * a[1, 1]
    m[2, 1] = a[2, 0] * a[0, 1] - a[0, 0] * a[2, 1]
    m[2, 2] = a[0, 0] * a[1, 1] - a[1, 0] * a[0, 1]
    return m * invdet


def _compute_cov_owr(a, su, sz, si, spd, spo):
    # compute covariance matrices
    s_n = np.array([[su ** 2 + sz ** 2, a ** 0.5 * si * su / 2, -a ** 0.5 * si * su / 2],
                    [a ** 0.5 * si * su / 2, spd ** 2 + a * si ** 2 / 4, -a * si ** 2 / 4],
                    [-a ** 0.5 * si * su / 2, -a * si ** 2 / 4, spo ** 2 + (1 + a) * si ** 2 / 4]])

    s_e = np.array([[(1 + 1 / a) * su ** 2 + sz ** 2, a ** (-0.5) * si * su / 2 + a ** 0.5 * si * su / 2,
                     a ** (-0.5) * si * su / 2 - a ** 0.5 * si * su / 2],
                    [a ** (-0.5) * si * su / 2 + a ** 0.5 * si * su / 2, spd ** 2 + (1 + a) * si ** 2 / 4,
                     (1 - a) * si ** 2 / 4],
                    [a ** (-0.5) * si * su / 2 - a ** 0.5 * si * su / 2, (1 - a) * si ** 2 / 4,
                     spo ** 2 + (1 + a) * si ** 2 / 4]])

    return s_n, s_e


def _compute_alpha_owr(x, a, dsd, isd):
    alphas = np.zeros(x.shape[1])
    for i in range(x.shape[1]):
        alphas[i] = 1 / (1 + (1 - a) / a * exp(_lf_owr(x[:, i], dsd, isd)))
    return alphas


def _lf_owr(x, det, inv):
    return -0.5 * log(det) - 0.5 * _qvmv_owr(x, inv)


@jit
def _qvmv_owr(x, A):
    """Computes x'Ax.
    """
    m, n = A.shape
    qsum = 0

    for i in range(m):
        for j in range(n):
            qsum += A[i, j] * x[i] * x[j]

    return qsum


def compute_alpha_owr(oib, ret_d, ret_o, a, su, sz, si, spd, spo):
    """Computes conditional alpha.
    
    Params
    ------
    oib : pandas.Series or numpy.ndarray
        Order imbalance, representing the net difference between buy and sell orders.
    ret_d : pandas.Series or numpy.ndarray
        Returns calculated during the day.
    ret_o : pandas.Series or numpy.ndarray
        Overnight returns.
    a : float
        Unconditional probability of an information event.
    su : float
        Standard deviation of uninformed trades.
    sz : float
        Standard deviation of noise trades.
    si : float
        Standard deviation of informed trades.
    spd : float
        Standard deviation of daily price changes.
    spo : float
        Standard deviation of overnight price changes.
    
    Returns
    -------
    pandas.Series
        Conditional probability (`alpha`) of an information event occurring for each time period.
    
    """
    # if len(a) > 1:
    #     a = a.tolist().pop()
    #     su = su.tolist().pop()
    #     sz = sz.tolist().pop()
    #     si = si.tolist().pop()
    #     spd = spd.tolist().pop()
    #     spo = spo.tolist().pop()
    # s_n, s_e = _compute_cov_owr(a, su, sz, si, spd, spo)
    # dsn, isn = det3_owr(s_n), inv3_owr(s_n)
    # dse, ise = det3_owr(s_e), inv3_owr(s_e)
    # dsd = dsn / dse
    # isd = isn - ise
    # 
    # x = np.array([oib, ret_d, ret_o])
    # cpie = pd.Series(_compute_alpha_owr(x, a, dsd, isd), index=oib.index)
    # return cpie
    s_n, s_e = _compute_cov_owr(a, su, sz, si, spd, spo)
    dsn, isn = det3_owr(s_n), inv3_owr(s_n)
    dse, ise = det3_owr(s_e), inv3_owr(s_e)
    dsd = dsn/dse
    isd = isn-ise
    
    x = np.array([oib, ret_d, ret_o])
    cpie = pd.Series(_compute_alpha_owr(x, a, dsd, isd), index=oib.index)
    return cpie[0]


In [129]:
a, su, sz, si, spd, spo = 0.780775, 0.188613492, 0.47896027, 0.022607443, 0.005858138, 0.01009005
ro, rd, o = -0.00662909, -0.013827281, -0.352941176

alpha_result = compute_alpha_owr(pd.Series([o]), pd.Series([rd]), pd.Series([ro]), a, su, sz, si, spd, spo)
print("Computed alpha:", alpha_result)

Computed alpha: 0    0.848057
dtype: float64


In [16]:
# Example usage to download data from WRDS TAQ database using WRDS package
def download_taq_data(start_year, end_year):
    """Downloads required data from WRDS TAQ database for calculating r_d, r_o, and y_e.
    
    Params
    ------
    start_year : int
        The starting year for the data download.
    end_year : int
        The ending year for the data download.
    
    Returns
    -------
    DataFrame
        A DataFrame containing the data needed to calculate r_d, r_o, and y_e.
    """
    db = wrds.Connection(wrds_username='aheitz')
    data_list = []

    for year in range(start_year, end_year + 1):
        query = f"""
        SELECT date, sym_root, sym_suffix, buynumtrades_lr, sellnumtrades_lr, oprc, cprc, ret_mkt_m,
               vw_price_m, mid_after_open, total_vol_m, total_vol_b, total_vol_a
        FROM taqmsec.wrds_iid_{year}
        WHERE sym_root IS NOT NULL
        """
        yearly_data = db.raw_sql(query)
        data_list.append(yearly_data)

    db.close()
    taq_data = pd.concat(data_list, ignore_index=True)
    return taq_data

In [2]:
db = wrds.Connection(wrds_username='aheitz')

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
db.close()

In [55]:
describe_table = db.describe_table('taqmsec', 'wrds_iid_2003')

Approximately 602729 rows in taqmsec.wrds_iid_2003.


In [27]:
test_taq_df3 = download_taq_data(2014, 2014)


Loading library list...
Done


In [18]:
test_taq_df = download_taq_data(2008, 2012)

OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: FATAL:  PAM authentication failed for user "aheitz"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [23]:
test_taq_df

(4070283, 13)

In [20]:
test_taq_df2 = download_taq_data(2017, 2019)

Loading library list...
Done


In [28]:
taq_df_14_19 = pd.concat([test_taq_df3, test_taq_df, test_taq_df2], ignore_index=True)

In [4]:
from Constants import Constants as const
import os

table_list = db.list_tables('taqmsec')
with open(os.path.join(const.TEMP_PATH, 'taqmsec_tabls.txt'), 'w') as f:
    for table in table_list:
        f.write(f"{table}\n")

In [30]:
data_list = list()

In [31]:
for year in range(2010, 2013):
    query = f"""
    SELECT date, sym_root, sym_suffix, buynumtrades_lr, sellnumtrades_lr, oprc, cprc, ret_mkt_m,
           vw_price_m, mid_after_open, total_vol_m, total_vol_b, total_vol_a
    FROM taqmsec.wrds_iid_{year}
    WHERE sym_root IS NOT NULL
    """
    yearly_data = db.raw_sql(query)
    data_list.append(yearly_data)

ProgrammingError: (psycopg2.errors.InsufficientPrivilege) permission denied for schema taqm_2010

[SQL: 
    SELECT date, sym_root, sym_suffix, buynumtrades_lr, sellnumtrades_lr, oprc, cprc, ret_mkt_m,
           vw_price_m, mid_after_open, total_vol_m, total_vol_b, total_vol_a
    FROM taqmsec.wrds_iid_2010
    WHERE sym_root IS NOT NULL
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [63]:
wrds_iid_df = pd.concat(data_list, ignore_index=True)
wrds_iid_df.to_pickle(os.path.join(const.TEMP_PATH, 'wrds_iid_2014_2016.pkl'))

In [32]:
wrds_iid_2013 = pd.read_pickle(os.path.join(const.TEMP_PATH, 'wrds_iid_2013.pkl'))
taq_data = pd.concat([wrds_iid_2013, taq_df_14_19], ignore_index=True)

# Calculating y_e, r_d, and r_o
taq_data['y_e'] = (taq_data['buynumtrades_lr'] - taq_data['sellnumtrades_lr']) / (taq_data['buynumtrades_lr'] + taq_data['sellnumtrades_lr'])
taq_data['r_d'] = (taq_data['vw_price_m'] - taq_data['mid_after_open'] + taq_data.get('divamt', 0)) / taq_data['mid_after_open']
taq_data['r_o'] = (taq_data['mid_after_open'] - taq_data['vw_price_m']) / taq_data['mid_after_open']

print(taq_data.head())

         date sym_root sym_suffix  buynumtrades_lr  sellnumtrades_lr   oprc  \
0  2013-01-02        A       None          13885.0           15996.0  41.88   
1  2013-01-02       AA       None          25249.0           24626.0   8.88   
2  2013-01-02       AA         PR              1.0               1.0  84.79   
3  2013-01-02     AACC       None            327.0             367.0   4.69   
4  2013-01-02     AADR       None              3.0               2.0  31.65   

    cprc  ret_mkt_m  vw_price_m  mid_after_open  total_vol_m  total_vol_b  \
0  41.88        0.0   41.575585           41.88    6264024.0      14270.0   
1   8.99   0.012311    8.915031           8.875   20558520.0     357248.0   
2   84.0  -0.009361   84.263333           84.42        300.0         <NA>   
3   4.84   0.031482    4.845709           4.915     113543.0         <NA>   
4   31.5  -0.004751   31.527453          31.485       2159.0         <NA>   

   total_vol_a       y_e       r_d       r_o  
0      12480.0 

In [14]:
gpin_parameter_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'gpin-1319.csv')).drop(['f', 'rc'], axis=1).rename(
    columns={'yyyy': 'year'})
gpin_parameter_df2: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'gpin_yearly.csv')).rename(
    columns={'yyyy': 'year'})
gpin_parameter_df: DataFrame = pd.concat([gpin_parameter_df, gpin_parameter_df2], ignore_index=True)
# owr_parameter_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'owr-1319.csv')).drop(['f', 'rc'], axis=1).rename(
#     columns={'yyyy': 'year'})

In [15]:
gpin_parameter_df.describe()

,permno,year,a,d,eta,p,r,th
count,36974.000000,36974.000000,36974.000000,36974.000000,36974.000000,36974.000000,36974.000000,36974.000000
mean,54209.762103,2008.107968,0.359440,0.525476,0.390450,0.954646,7.065471,0.498195
std,28471.608336,8.133014,0.198814,0.166802,0.344609,0.096534,8.712734,0.059621
min,10001.000000,1993.000000,0.000010,0.000010,0.000010,0.019794,0.245636,0.000010
25%,24096.000000,2001.000000,0.171945,0.417950,0.119072,0.966017,3.824508,0.487250
50%,58990.000000,2010.000000,0.411983,0.501555,0.237896,0.995091,5.680068,0.501481
75%,80531.500000,2015.000000,0.518584,0.621032,0.651406,0.999016,8.629960,0.520459
max,93436.000000,2019.000000,0.999990,0.999990,20.548875,0.999990,985.821894,0.999990


In [36]:
query = f"""
SELECT date, permno, sym_suffix, sym_root
FROM wrdsapps_link_crsp_taqm.taqmclink
WHERE sym_root IS NOT NULL AND date between '2013-01-01' and '2019-12-31'
"""
taq_crsp_link = db.raw_sql(query)

In [37]:
db.close()

In [38]:
taq_crsp_link.to_pickle(os.path.join(const.TEMP_PATH, 'taq_crsp_linkage.pkl'))

In [39]:
taq_crsp_link.memory_usage(True).sum() /(1024 * 1024)

344.9998664855957

In [63]:
taq_crsp_link.head()

,date,permno,sym_suffix,sym_root
0,2013-01-02,87432,<NA>,A
1,2013-01-03,87432,<NA>,A
2,2013-01-04,87432,<NA>,A
3,2013-01-07,87432,<NA>,A
4,2013-01-08,87432,<NA>,A


In [64]:
taq_data.shape

(14217482, 16)

In [66]:
taq_data_crsp = taq_data.drop(['sym_suffix'], axis=1).drop_duplicates(
    subset=['date', 'sym_root'], keep='first').merge(
    taq_crsp_link.dropna(subset=['permno']).drop(['sym_suffix'], axis=1).drop_duplicates(
    subset=['date', 'sym_root'], keep='first'), on=['date', 'sym_root'], how='left')
taq_data_crsp.shape

(13080717, 16)

In [67]:
taq_data_crsp['permno'] = taq_data_crsp.groupby(['sym_root'])['permno'].bfill()
taq_data_crsp['permno'] = taq_data_crsp.groupby(['sym_root'])['permno'].ffill()
taq_data_crsp.dropna(subset=['permno'], inplace=True)
taq_data_crsp.shape

(8907055, 16)

In [68]:
taq_data_crsp['date'] = pd.to_datetime(taq_data['date'])
taq_data_crsp['year'] = taq_data_crsp['date'].dt.year
taq_data_crsp['permno'] = pd.to_numeric(taq_data_crsp['permno'], errors='coerce')
taq_data_crsp_model = taq_data_crsp.merge(gpin_parameter_df, on=['permno', 'year'], how='left').replace({'NA': np.nan})

In [42]:
taq_data_crsp_model.keys()

Index(['date', 'sym_root', 'buynumtrades_lr', 'sellnumtrades_lr', 'oprc',
       'cprc', 'ret_mkt_m', 'vw_price_m', 'mid_after_open', 'total_vol_m',
       'total_vol_b', 'total_vol_a', 'y_e', 'r_d', 'r_o', 'permno', 'year',
       'a', 'd', 'eta', 'p', 'r', 'th', 'a_owr', 'si', 'spd', 'spo', 'su',
       'sz'],
      dtype='object')

In [69]:
taq_data_crsp_model.dropna(subset=['a', 'd', 'eta', 'p', 'r', 'th', 'buynumtrades_lr'], how='any', inplace=True)

In [52]:
from tqdm.notebook import tqdm

tqdm.pandas()

In [70]:
taq_data_crsp_model.loc[:, 'gpin'] = taq_data_crsp_model.progress_apply(
    lambda x: compute_alpha_gpin(x['a'], x['r'], x['p'], x['eta'], x['d'], x['th'], x['buynumtrades_lr'], x['sellnumtrades_lr']), axis=1)

  0%|          | 0/3840389 [00:00<?, ?it/s]

In [71]:
taq_data_crsp_model.iloc[1950]

date                2013-01-02 00:00:00
sym_root                           SONS
buynumtrades_lr                  8244.0
sellnumtrades_lr                 4491.0
oprc                               1.78
cprc                               1.95
ret_mkt_m                      0.091216
vw_price_m                     1.853086
mid_after_open                     1.74
total_vol_m                   5561178.0
total_vol_b                     20500.0
total_vol_a                     54820.0
y_e                              0.2947
r_d                            0.064992
r_o                           -0.064992
permno                            88264
year                               2013
a                              0.546487
d                              0.419592
eta                            0.195658
p                              0.999065
r                              4.881155
th                             0.503015
gpin                                1.0
Name: 3639, dtype: object

In [57]:
row = taq_data_crsp_model.iloc[1950]
compute_alpha_gpin(row['a'], row['r'], row['p'], row['eta'], row['d'], row['th'], row['buynumtrades_lr'], row['sellnumtrades_lr'])

TypeError: boolean value of NA is ambiguous

In [130]:
taq_data_crsp_model.loc[:, 'owr'] = taq_data_crsp_model.apply(
    lambda x: compute_alpha_owr(pd.Series([x['y_e']]), pd.Series([x['r_d']]), pd.Series([x['r_o']]), x['a_owr'], x['su'], x['sz'], x['si'], x['spd'], x['spo']), axis=1)

C:\Users\wangy\AppData\Local\Temp\ipykernel_23904\3575950180.py:42: RuntimeWarning: overflow encountered in exp
  alphas[i] = 1 / (1 + (1 - a) / a * exp(_lf_owr(x[:, i], dsd, isd)))
C:\Users\wangy\AppData\Local\Temp\ipykernel_23904\3575950180.py:42: RuntimeWarning: overflow encountered in scalar multiply
  alphas[i] = 1 / (1 + (1 - a) / a * exp(_lf_owr(x[:, i], dsd, isd)))


In [72]:
taq_data_crsp_model.to_pickle(os.path.join(const.TEMP_PATH, '2013_2019_gpin.pkl'))

In [5]:
taq_data_crsp_model = pd.read_pickle(os.path.join(const.TEMP_PATH, '2013_2016_gpin_owr.pkl'))

In [10]:
taq_data_crsp_model.head()

,date,sym_root,buynumtrades_lr,sellnumtrades_lr,oprc,cprc,ret_mkt_m,vw_price_m,mid_after_open,total_vol_m,...,r,th,a_owr,si,spd,spo,su,sz,gpin,owr
0,2013-01-02,A,13885.0,15996.0,41.88,41.8800,0.000000,41.575585,41.880,6264024.0,...,5.586394,0.505736,0.999990,0.003834,0.006773,0.006000,0.028366,0.044506,1.000000e+00,0.999989
1,2013-01-02,AA,25249.0,24626.0,8.88,8.9900,0.012311,8.915031,8.875,20558520.0,...,6.382776,0.497251,0.701706,0.011844,0.004628,0.000010,0.025246,0.091121,8.106653e-93,0.591216
5,2013-01-02,AAME,9.0,14.0,3.09,3.1999,0.034948,3.168289,3.190,3700.0,...,2.034722,0.461257,0.694292,0.030881,0.007788,0.000010,0.085575,0.404290,1.065108e-02,0.568964
6,2013-01-02,AAN,1901.0,1607.0,28.72,28.7000,-0.000697,28.527780,28.730,454686.0,...,6.100579,0.497322,0.880591,0.005125,0.007729,0.005901,0.068434,0.010389,9.999848e-01,0.932259
8,2013-01-02,AAP,2591.0,2443.0,73.58,72.4900,-0.014925,72.460757,73.495,754331.0,...,4.295786,0.505725,0.749298,0.010862,0.003318,0.000010,0.020013,0.081053,1.048363e-09,0.460071


In [11]:
quarter_gpin_owr_link: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, 'cpin_gpin_owr_quarterly.pkl'))

# Construct Price Delay
Hou, K., & Moskowitz, T. J. (2005). Market Frictions, Price Delay, and the Cross-Section of Expected Returns. The Review of Financial Studies, 18(3), 981–1020. https://doi.org/10.1093/rfs/hhi023


In [4]:
import os

import pandas as pd
import numpy as np
import statsmodels.api as sm

from Constants import Constants as const

In [5]:
# df = pd.read_csv(os.path.join(const.DATABASE_PATH, 'crsp', '2007_2015_CRSP_daily_ret.zip'), dtype={"NCUSIP": str, 'date': str})
data_path = r'D:\Users\wangy\Documents\data'

# Load the CRSP data
df = pd.read_csv(os.path.join(data_path, '20010101_20161231_crsp_stock_return.zip'), dtype={"NCUSIP": str, 'date': str})


# Step 1: 基础处理
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['PERMNO', 'date'])
df['year'] = df['date'].dt.year
df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
df['vwretd'] = pd.to_numeric(df['vwretd'], errors='coerce')
df = df.dropna(subset=['RET', 'vwretd'])

# Step 2: 转换为 weekly compounded returns
weekly = (
    df.set_index('date')
      .groupby('PERMNO')[['RET', 'vwretd']]
      .resample('W-FRI')
      .apply(lambda x: (1 + x).prod() - 1)
      .dropna()
      .reset_index()
)

# 添加年份变量（用于 firm-year）
weekly['year'] = weekly['date'].dt.year

# Step 3: 对每个 firm-year 执行回归并计算 price delay
results = []

grouped = weekly.groupby(['PERMNO', 'year'])
for (permno, year), group in grouped:
    group = group.sort_values('date').reset_index(drop=True)

    # 构造 market return 滞后变量
    for lag in range(1, 5):
        group[f'vwretd_lag{lag}'] = group['vwretd'].shift(lag)

    group = group.dropna()
    if len(group) < 10:
        continue  # 跳过样本太少的 firm-year

    y = group['RET'].values
    X_full = sm.add_constant(group[['vwretd', 'vwretd_lag1', 'vwretd_lag2', 'vwretd_lag3', 'vwretd_lag4']].values)
    X_restricted = sm.add_constant(group[['vwretd']].values)

    model_full = sm.OLS(y, X_full).fit()
    model_restricted = sm.OLS(y, X_restricted).fit()

    r2_full = model_full.rsquared
    r2_restricted = model_restricted.rsquared

    delay = 1 - r2_restricted / r2_full if r2_full > 0 else np.nan

    results.append({
        'PERMNO': permno,
        'year': year,
        'price_delay': delay,
        'r2_full': r2_full,
        'r2_restricted': r2_restricted,
        'n_obs': len(group)
    })

# Step 4: 输出 firm-year 结果表格
price_delay_df = pd.DataFrame(results)


C:\Users\wangy\AppData\Local\Temp\ipykernel_35168\1587979423.py:5: DtypeWarning: Columns (2,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_path, '20010101_20161231_crsp_stock_return.zip'), dtype={"NCUSIP": str, 'date': str})
D:\anaconda3\envs\finnlp\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
D:\anaconda3\envs\finnlp\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
D:\anaconda3\envs\finnlp\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
D:\anaconda3\envs\finnlp\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar div

In [9]:
price_delay_df.head()

,PERMNO,year,price_delay,r2_full,r2_restricted,n_obs
0,10001,2007,0.487723,0.128451,0.065803,48
1,10001,2008,0.079623,0.470354,0.432903,48
2,10001,2009,0.853172,0.110732,0.016259,48
3,10001,2010,0.851495,0.316308,0.046973,49
4,10001,2011,0.369159,0.079426,0.050105,48


In [6]:
price_delay_df.to_pickle(os.path.join(const.TEMP_PATH, 'hm_2005_price_delay.pkl'))